In [1]:
from __future__ import division, print_function, absolute_import
import os

import tflearn
from tflearn.data_utils import shuffle, to_categorical
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression
from tflearn.data_preprocessing import ImagePreprocessing
from tflearn.data_augmentation import ImageAugmentation


hdf5 is not supported on this machine (please install/reinstall h5py for optimal experience)


In [2]:
# Data loading and preprocessing
import numpy as np
datadir = '/home/marysia/thesis/data/'
data = np.load(datadir+'KNEE/train.npz')
X = data['data']
Y = data['labels']
data = np.load(datadir+'KNEE/test.npz')
X_test = data['data']
Y_test = data['labels']

Y = to_categorical(Y, 2)
Y_test = to_categorical(Y_test, 2)
print(X.shape)
print(Y.shape)

print('Loaded.')

(15000, 1, 100, 200)
(15000, 2)
Loaded.


In [3]:
# Real-time data preprocessing
img_prep = ImagePreprocessing()
img_prep.add_featurewise_zero_center()
img_prep.add_featurewise_stdnorm()

# Real-time data augmentation
img_aug = ImageAugmentation()
img_aug.add_random_flip_leftright()
img_aug.add_random_rotation(max_angle=25.)
print('Augmented.')

Augmented.


In [4]:
# Convolutional network building
#network = input_data(shape=[None, 1, 100, 200],
                     #data_preprocessing=img_prep,
                     #data_augmentation=img_aug)
network = input_data(shape=[None, 1, 100, 200])
network = conv_2d(network, 32, 3, activation='relu')
network = max_pool_2d(network, 2)
network = conv_2d(network, 64, 3, activation='relu')
network = conv_2d(network, 64, 3, activation='relu')
network = max_pool_2d(network, 2)
network = fully_connected(network, 512, activation='relu')
network = dropout(network, 0.5)
network = fully_connected(network, 2, activation='softmax')
network = regression(network, optimizer='adam',
                     loss='categorical_crossentropy',
                     learning_rate=0.001)
print('Network created.')

Network created.


In [5]:
# Train using classifier
model = tflearn.DNN(network, tensorboard_verbose=0)
model.fit(X, Y, n_epoch=10, shuffle=True, validation_set=(X_test, Y_test),
          show_metric=True, batch_size=96, run_id='knee_cnn')

Training Step: 1569  | total loss: 0.07593 | time: 3.477s
| Adam | epoch: 010 | loss: 0.07593 - acc: 0.9896 -- iter: 14976/15000
Training Step: 1570  | total loss: 0.06845 | time: 8.477s
| Adam | epoch: 010 | loss: 0.06845 - acc: 0.9906 | val_loss: 0.01062 - val_acc: 0.9967 -- iter: 15000/15000
--


In [6]:
print(model.evaluate(X_test, Y_test))

[0.99674960805866863]
